In [ ]:
public static MirAnimation GetMagicAnimation(MagicType m)  //获得魔法攻击动画
{
    switch (m)
    {
        ....
        case MagicType.Asteroid:
            return MirAnimation.Combat2;
        ....
    }
}

In [ ]:
public virtual void SetAction(ObjectAction action)
{
    MirEffect spell;
    
    switch (CurrentAction)
    {
    ....
    case MirAction.Spell:
        if (!MagicCast)
        {
            break;
        }

        switch (MagicType)
        {
        ....
        case MagicType.Asteroid://天之怒火
        if (!Config.DrawEffects) break;
        foreach (Point point in MagicLocations)
        {
            MirProjectile eff;
            Point p = new Point(point.X + 4, point.Y - 10);
            Effects.Add(eff = new MirProjectile(1300, 10, TimeSpan.FromMilliseconds(100), LibraryFile.MagicEx5, 50, 80, Globals.FireColour, p)
            {
                MapTarget = point,
                Skip = 0,
                DrawType = DrawType.FlyMagic,
                Explode = true,
                Blend = true,
            });
    
            eff.CompleteAction = () =>
            {
                Effects.Add(new MirEffect(1320, 8, TimeSpan.FromMilliseconds(100), LibraryFile.MagicEx5, 100, 100, Globals.NoneColour)
                {
                    MapTarget = eff.MapTarget,
                    Blend = true,
                });
            };
        }
        break;

In [ ]:
 public bool UseMagic(ClientUserMagic magic)
 {
     if (CEnvir.Now < NextMagicTime) return false;
    //if (CEnvir.Now < NextActionTime) return false;
    
    if (magic == null || Level < magic.Info.NeedLevel1 || magic.NextCast > CEnvir.Now) return false;
    
    bool CheckRange = false;
     
    ....
        
    switch (magic.Info.Magic)
    
    ....
        
    switch (magic.Info.Magic)
    
    ....
        
    MirDirection direction = Game.MapControl.MouseDirection();

    switch (magic.Info.Magic)
    {
    case MagicType.ShoulderDash://野蛮冲撞
        if (CEnvir.Now < ServerTime) return false;
        if ((Poison & PoisonType.WraithGrip) == PoisonType.WraithGrip) return false;

        ServerTime = CEnvir.Now.AddSeconds(5);
        NextMagicTime = CEnvir.Now + Globals.MagicDelay;
        CEnvir.Enqueue(new C.Magic { Direction = direction, Action = MirAction.Spell, Type = magic.Info.Magic });
        return true;
    ....
    case MagicType.Asteroid://天之怒火
    case MagicType.MeteorShower://焰天火雨
    case MagicType.Tempest://旋风墙
    case MagicType.MagicResistance://幽灵盾
    case MagicType.Resilience://神圣战甲
    case MagicType.LifeSteal://吸星大法
    case MagicType.MassInvisibility://集体隐身术
    case MagicType.TrapOctagon://困魔咒
    case MagicType.ElementalSuperiority://强魔震法
    case MagicType.BloodLust://猛虎强势
    case MagicType.MassHeal://群体治愈术
        CheckRange = true;
        break;
    default:
        return false;
    }

    if (CheckRange && (target != null && !Functions.InRange(target.CurrentLocation, CurrentLocation, Globals.MagicRange) || (target == null && !Functions.InRange(Game.MapControl.MapLocation, CurrentLocation, Globals.MagicRange))))
    {
        if (CEnvir.Now < OutputTime) return false;
        OutputTime = CEnvir.Now.AddSeconds(1);
        Game.ReceiveChat($"不能使用 {magic.Info.Name}, 你的攻击目标太远了", MessageType.Hint);
        return false;
    }

In [ ]:
private bool CheckExcepted(bool player)
{
    switch (Magic)
    {
    case MagicType.Purification://云寂术
    case MagicType.MeteorShower://炎天火雨
    case MagicType.Resurrection://回生术
    case MagicType.Asteroid://天之怒火
        return true;
    case MagicType.Interchange:
    case MagicType.Beckon:
        return player;
    default:
        break;
    }
    return false;
}

In [ ]:
protected internal override void OnChanged(object oldValue, object newValue, string propertyName)
{
    switch(propertyName)
    {
    case "Magic":
        {
            ....
                
            //勾选状态特定值
            switch (Magic)
            {
            case MagicType.Purification://云寂术
            case MagicType.MeteorShower://炎天火雨
            case MagicType.Asteroid://天之怒火
                LockMonster = false;
                LockPlayer = false;
                break;
            }
            ....
        }
        ....
    }
    ....
}

In [ ]:
public int MagicAttack(List<UserMagic> magics, MapObject ob, bool primary/*是否中心目标（群体伤害）*/, Stats stats = null, int extra = 0)
{
    if (ob?.Node == null || ob.Dead) return 0;
    
    ....
        
    int power = 0;
    UserMagic asteroid = null;
    MagicType mtype = MagicType.None;
    foreach (UserMagic magic in magics)
    {
        mtype = magic.Info.Magic;
        switch (mtype)
        {
        case MagicType.FireBall:
        case MagicType.ScortchedEarth:
        case MagicType.FireStorm:
        case MagicType.AdamantineFireBall:
            element = Element.Fire;
            power += magic.GetPower() + GetMC();
            break;
        ....
        case MagicType.Asteroid://天之怒火伤害计算
            element = Element.Fire;
            //新改
            power += magic.GetPower() + GetMC();
    
            asteroid = magic;
            canStuck = false;
            break;
        ....
    }

    ....
        
    epower = (int)(Config.AttackElement * epower);

    //强弱元素计算  dpower<0 弱元素
    power = (int)(epower - epower * dpower * (dpower>0?0.17:0.30) + (power - 0.08 * dpower * power) * ( (element>=Element.Holy) ? Config.DefenceSCElement: Config.DefenceMCElement));
    //如果power100 弱5 =>100+(0.08*5*100)
    
    // 绕过强元素 后处理
    //if (asteroid != null)
    //{
    //    //天之怒火 额外伤害
    //    power += asteroid.GetPower() + GetMC();
    //}
    
    if (power <= 0)
    {
        ob.Blocked();
        return 0;
    }

    ....
        
}

In [ ]:
public void Magic(C.Magic p)  //技能
{
    ....
    switch (p.Type)
    {
    case MagicType.Throw:
    case MagicType.ShoulderDash:
    ....
    case MagicType.Teleportation:
    case MagicType.Asteroid://天之怒火
        if (magic.Cost > CurrentMP)
        {
            Enqueue(new S.UserLocation { Direction = Direction, Location = CurrentLocation });
            return;
        }
        break;
    ....
    }
    ....
    int count;
    List<MapObject> possibleTargets;
    Point location;
    BuffInfo buff;
    switch (p.Type)
    {
    case MagicType.ShoulderDash:
        if ((Poison & PoisonType.WraithGrip) == PoisonType.WraithGrip) break;//石化
    
        Direction = p.Direction;
        count = ShoulderDashEnd( magic );
    
        if (count == 0)
        {
            Connection.ReceiveChat( Connection.Language.DashFailed , MessageType.System );
    
            foreach (SConnection con in Connection.Observers)
                con.ReceiveChat( con.Language.DashFailed , MessageType.System );
        }
    
        Enqueue( new S.MagicCooldown { InfoIndex = magic.Info.Index , Delay = magic.Info.Delay } );
        magic.Cooldown = SEnvir.Now.AddMilliseconds( magic.Info.Delay );
        ChangeMP( -magic.Cost );
        return;
    case MagicType.Asteroid:  //天之怒火
        ob = null;
        if (!Functions.InRange( CurrentLocation , p.Location , Globals.MagicRange ))
        {
            cast = false;
            break;
        }
    
        locations.Add( p.Location );
        cells = CurrentMap.GetCells( p.Location , 0 , 2 );
    
        foreach (Cell cell in cells)
        {
            ActionList.Add( new DelayedAction(
                SEnvir.Now.AddMilliseconds( 1200 ) ,
                ActionType.DelayMagic ,
                new List<UserMagic> { magic } ,
                cell ) );
        }
    
    
        if (Magics.TryGetValue( MagicType.FireWall , out augMagic ) && augMagic.Info.NeedLevel1 > Level)
            augMagic = null;
    
    
        if (augMagic != null)
        {
            foreach (ConquestWar war in SEnvir.ConquestWars)
            {
                if (war.Map != CurrentMap) continue;
    
                for (int i = SpellList.Count - 1 ; i >= 0 ; i--)
                {
                    if (SpellList [ i ].Effect != SpellEffect.FireWall) continue;
    
                    SpellList [ i ].Despawn();
                }
    
                break;
            }
    
            power = (magic.Level + 2) * 5;
    
            foreach (Cell cell in cells)
            {
    
                if (Math.Abs( cell.Location.X - p.Location.X ) + Math.Abs( cell.Location.Y - p.Location.Y ) >= 3) continue;
    
    
                ActionList.Add( new DelayedAction(
                    SEnvir.Now.AddMilliseconds( 2800 ) ,
                    ActionType.DelayMagic ,
                    new List<UserMagic> { augMagic } ,
                    cell ,
                    power ) );
            }
        }
    
        break;
    }
    ....
    }

}

In [ ]:
public void CompleteMagic(params object[] data)
{
    List<UserMagic> magics = (List<UserMagic>)data[0];
    foreach (UserMagic magic in magics)
    {

        switch (magic.Info.Magic)
        {
        ....
        case MagicType.Asteroid:
            AttackCell(magics, (Cell)data[1], true);
            break;
        ....
        }
        ....
    }
    ....
}